In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"
# %pip install "unsloth[cu121_torch220] @ git+https://github.com/unslothai/unsloth.git"

In [1]:
from lora import Lora_FineTuner

In [2]:
bucket='llm-storage-bucket'
data_key = 'Cosmetics_dataset_0.8/'
dataset_path = 's3://{}/{}'.format(bucket, data_key)

# dataset_path = 'Cosmetics_dataset_0.8/'

In [3]:
llm_trainer = Lora_FineTuner(dataset=dataset_path,
                             # model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
                             model_name = "unsloth/mistral-7b-v0.2-bnb-4bit",
                             lora_rank=64,
                             rs_lora=True,
                             max_seq_length=140
                             )

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
llm_trainer.train(n_epochs=3, 
                  device_batch_size=16, 
                  save_steps= 50,
                  eval_steps=50,
                  resume = False,
                  # n_rows=250,
                 )

Map (num_proc=2):   0%|          | 0/9436 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2442 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,436 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 441
 "-____-"     Number of trainable parameters = 167,772,160
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: goutham_city. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
50,1.106400,1.015433


### Saving for VLLM

In [5]:
import datetime
now = datetime.datetime.now()

In [6]:
model_save_path_16bit = f"Cosmetics_Expert_{now.strftime('%Y-%m-%d_%H%M%S')}"

In [7]:
llm_trainer.model.save_pretrained_merged(model_save_path_16bit, llm_trainer.tokenizer, save_method = "merged_16bit",)

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 8.69 out of 15.32 RAM for saving.


100%|██████████| 32/32 [00:49<00:00,  1.55s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving Cosmetics_Expert_2024-03-25_162452/pytorch_model-00001-of-00003.bin...
Unsloth: Saving Cosmetics_Expert_2024-03-25_162452/pytorch_model-00002-of-00003.bin...
Unsloth: Saving Cosmetics_Expert_2024-03-25_162452/pytorch_model-00003-of-00003.bin...
Done.


### Upload to S3

In [8]:
import boto3
import os

In [9]:
session = boto3.Session()

In [10]:
models_key = f"Models/{model_save_path_16bit}/"
model_to_upload = f"./{model_save_path_16bit}/"

In [11]:
for file in os.listdir(model_to_upload):
    file_path = os.path.join(model_to_upload, file)
    if os.path.isfile(file_path):
        # Upload the file to S3
        session.resource('s3').Bucket(bucket).Object(os.path.join(models_key, file)).upload_file(file_path)


In [ ]:
1+1